In [1]:
import time
import os
import sys

sys.path.append('model')
from tqdm import tqdm

import torch
from torch import cuda
from model.data_loaders import LaneDataSet
from model.transformers import Rescale
from model.lanenet import LaneNet, compute_loss
from torch.utils.data import DataLoader
from torch.autograd import Variable

from torchvision import transforms

import numpy as np
import cv2

# might want this in the transformer part as well
VGG_MEAN = [103.939, 116.779, 123.68]
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda:0


In [2]:
def compose_img(image_data, out, binary_label, pix_embedding, instance_label, i):
    val_gt = (image_data[i].cpu().numpy().transpose(1, 2, 0) + VGG_MEAN).astype(np.uint8)
    val_pred = out[i].squeeze(0).cpu().numpy().transpose(0, 1) * 255
    val_label = binary_label[i].squeeze(0).cpu().numpy().transpose(0, 1) * 255
    val_out = np.zeros((val_pred.shape[0], val_pred.shape[1], 3), dtype=np.uint8)
    val_out[:, :, 0] = val_pred
    val_out[:, :, 1] = val_label
    val_gt[val_out == 255] = 255
    epsilon = 1e-5
    pix_embedding = pix_embedding[i].data.cpu().numpy()
    pix_vec = pix_embedding / (np.sum(pix_embedding, axis=0, keepdims=True) + epsilon) * 255
    pix_vec = np.round(pix_vec).astype(np.uint8).transpose(1, 2, 0)
    ins_label = instance_label[i].data.cpu().numpy().transpose(0, 1)
    ins_label = np.repeat(np.expand_dims(ins_label, -1), 3, -1)
    val_img = np.concatenate((val_gt, pix_vec, ins_label), axis=0)
    return val_img

In [3]:
class AverageMeter():

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
def train(train_loader, model, optimizer, epoches,save_path):
    
    mean_iou = AverageMeter()
    total_losses = AverageMeter()
    binary_losses = AverageMeter()
    instance_losses = AverageMeter()
    
    
    for epoch in range(epoches):
        step = 0
        print(f"Epoch {epoch+1}")
        for batch_idx, batch in enumerate(train_loader):
            start = time.time()
            step += 1
            image_data = batch[0].type(torch.FloatTensor).to(DEVICE)
            binary_label = batch[1].type(torch.LongTensor).to(DEVICE)
            instance_label = batch[2].type(torch.FloatTensor).to(DEVICE)

            # forward pass
            net_output = model(image_data)

            # compute loss
            total_loss, binary_loss, instance_loss, out, train_iou = compute_loss(net_output, binary_label, instance_label,True)

            # update loss in AverageMeter instance
            total_losses.update(total_loss.item(), image_data.size()[0])
            binary_losses.update(binary_loss.item(), image_data.size()[0])
            instance_losses.update(instance_loss.item(), image_data.size()[0])
            mean_iou.update(train_iou, image_data.size()[0])

            # reset gradients
            optimizer.zero_grad()

            # backpropagate
            total_loss.backward()

            # update weights
            optimizer.step()

            # update batch time
            
            end = time.time()
            batch_time = end - start

            if step % 10 == 0:
                print(
                    "Epoch {ep} Step {st} |({batch}/{size})| ETA: {et:.2f}|Total:{tot:.5f}|Binary:{bin:.5f}|Instance:{ins:.5f}|IoU:{iou:.5f}".format(
                        ep=epoch + 1,
                        st=step,
                        batch=batch_idx + 1,
                        size=len(train_loader),
                        et=batch_time,
                        tot=total_losses.avg,
                        bin=binary_losses.avg,
                        ins=instance_losses.avg,
                        iou=train_iou,
                    ))
                sys.stdout.flush()
                # train_img_list = []
                # for i in range(3):
                #     train_img_list.append(
                #         compose_img(image_data, out, binary_label, net_output["instance_seg_logits"], instance_label, i))
                # train_img = np.concatenate(train_img_list, axis=1)
                # cv2.imwrite(os.path.join("./output", "train_" + str(epoch + 1) + "_step_" + str(step) + ".png"), train_img)
        if (epoch + 1) % 5 == 0:
            print("should save model")
            torch.save(model.state_dict(), os.path.join(save_path, ('lanenet_'+str(epoch+1)+'.pth')))
    return mean_iou.avg


In [11]:
def main():
    EPOCHES = 100
    file_dataset = './tusimple_0531/training'
    save_path = 'save_path'

    if not os.path.isdir(save_path):
        os.makedirs(save_path)

    train_dataset_file = os.path.join(file_dataset, 'train.txt')
    # val_dataset_file = os.path.join(file_dataset, 'val.txt')

    train_dataset = LaneDataSet(train_dataset_file, 
        transform=transforms.Compose([transforms.Resize((256,512)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.35248518, 0.37250566, 0.3955756], 
                std=[0.20043196, 0.2173182, 0.2510542])]))
    # val_dataset = LaneDataSet(val_dataset_file, transform=transforms.Compose([Rescale((512, 256))]))

    model = LaneNet()
    model.to(DEVICE)

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    # val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    print(f"{EPOCHES} epochs {len(train_dataset)} training samples\n")
    train(train_loader, model, optimizer, EPOCHES,save_path)

In [12]:
main()

100 epochs 358 training samples

Epoch 1
Epoch 1 Step 10 |(10/90)| ETA: 0.07|Total:0.19601|Binary:0.19302|Instance:0.00299|IoU:0.00000
Epoch 1 Step 20 |(20/90)| ETA: 0.07|Total:0.12506|Binary:0.12289|Instance:0.00217|IoU:0.00000
Epoch 1 Step 30 |(30/90)| ETA: 0.07|Total:0.09931|Binary:0.09741|Instance:0.00190|IoU:0.00000
Epoch 1 Step 40 |(40/90)| ETA: 0.07|Total:0.08672|Binary:0.08493|Instance:0.00179|IoU:0.00064
Epoch 1 Step 50 |(50/90)| ETA: 0.07|Total:0.07797|Binary:0.07627|Instance:0.00170|IoU:0.01949
Epoch 1 Step 60 |(60/90)| ETA: 0.07|Total:0.07220|Binary:0.07056|Instance:0.00164|IoU:0.00000
Epoch 1 Step 70 |(70/90)| ETA: 0.07|Total:0.06823|Binary:0.06663|Instance:0.00160|IoU:0.01704
Epoch 1 Step 80 |(80/90)| ETA: 0.08|Total:0.06465|Binary:0.06308|Instance:0.00157|IoU:0.01263
Epoch 1 Step 90 |(90/90)| ETA: 0.04|Total:0.06223|Binary:0.06068|Instance:0.00155|IoU:0.00000
Epoch 2
Epoch 2 Step 10 |(10/90)| ETA: 0.07|Total:0.06003|Binary:0.05850|Instance:0.00153|IoU:0.02992
Epoch 2 Ste

Epoch 10 Step 60 |(60/90)| ETA: 0.07|Total:0.03239|Binary:0.03102|Instance:0.00137|IoU:0.32798
Epoch 10 Step 70 |(70/90)| ETA: 0.07|Total:0.03230|Binary:0.03093|Instance:0.00137|IoU:0.21696
Epoch 10 Step 80 |(80/90)| ETA: 0.07|Total:0.03220|Binary:0.03083|Instance:0.00137|IoU:0.31570
Epoch 10 Step 90 |(90/90)| ETA: 0.05|Total:0.03211|Binary:0.03074|Instance:0.00137|IoU:0.40060
should save model
Epoch 11
Epoch 11 Step 10 |(10/90)| ETA: 0.07|Total:0.03201|Binary:0.03064|Instance:0.00137|IoU:0.32648
Epoch 11 Step 20 |(20/90)| ETA: 0.07|Total:0.03194|Binary:0.03057|Instance:0.00137|IoU:0.30198
Epoch 11 Step 30 |(30/90)| ETA: 0.07|Total:0.03187|Binary:0.03050|Instance:0.00137|IoU:0.32870
Epoch 11 Step 40 |(40/90)| ETA: 0.07|Total:0.03177|Binary:0.03040|Instance:0.00137|IoU:0.29095
Epoch 11 Step 50 |(50/90)| ETA: 0.07|Total:0.03169|Binary:0.03032|Instance:0.00137|IoU:0.39829
Epoch 11 Step 60 |(60/90)| ETA: 0.07|Total:0.03162|Binary:0.03026|Instance:0.00137|IoU:0.37411
Epoch 11 Step 70 |(70/9

Epoch 20 Step 10 |(10/90)| ETA: 0.07|Total:0.02742|Binary:0.02606|Instance:0.00136|IoU:0.40257
Epoch 20 Step 20 |(20/90)| ETA: 0.07|Total:0.02737|Binary:0.02601|Instance:0.00136|IoU:0.38140
Epoch 20 Step 30 |(30/90)| ETA: 0.07|Total:0.02734|Binary:0.02598|Instance:0.00136|IoU:0.37266
Epoch 20 Step 40 |(40/90)| ETA: 0.07|Total:0.02729|Binary:0.02593|Instance:0.00136|IoU:0.39886
Epoch 20 Step 50 |(50/90)| ETA: 0.07|Total:0.02725|Binary:0.02589|Instance:0.00136|IoU:0.45690
Epoch 20 Step 60 |(60/90)| ETA: 0.07|Total:0.02722|Binary:0.02586|Instance:0.00136|IoU:0.29471
Epoch 20 Step 70 |(70/90)| ETA: 0.07|Total:0.02718|Binary:0.02582|Instance:0.00136|IoU:0.37108
Epoch 20 Step 80 |(80/90)| ETA: 0.07|Total:0.02714|Binary:0.02578|Instance:0.00136|IoU:0.41827
Epoch 20 Step 90 |(90/90)| ETA: 0.04|Total:0.02711|Binary:0.02575|Instance:0.00136|IoU:0.36560
should save model
Epoch 21
Epoch 21 Step 10 |(10/90)| ETA: 0.07|Total:0.02707|Binary:0.02571|Instance:0.00136|IoU:0.31856
Epoch 21 Step 20 |(20/9

Epoch 29 Step 60 |(60/90)| ETA: 0.07|Total:0.02501|Binary:0.02365|Instance:0.00136|IoU:0.46804
Epoch 29 Step 70 |(70/90)| ETA: 0.07|Total:0.02498|Binary:0.02362|Instance:0.00136|IoU:0.44957
Epoch 29 Step 80 |(80/90)| ETA: 0.07|Total:0.02496|Binary:0.02360|Instance:0.00136|IoU:0.30775
Epoch 29 Step 90 |(90/90)| ETA: 0.04|Total:0.02494|Binary:0.02358|Instance:0.00136|IoU:0.46974
Epoch 30
Epoch 30 Step 10 |(10/90)| ETA: 0.07|Total:0.02491|Binary:0.02355|Instance:0.00136|IoU:0.48125
Epoch 30 Step 20 |(20/90)| ETA: 0.07|Total:0.02488|Binary:0.02353|Instance:0.00136|IoU:0.39887
Epoch 30 Step 30 |(30/90)| ETA: 0.07|Total:0.02486|Binary:0.02350|Instance:0.00136|IoU:0.55095
Epoch 30 Step 40 |(40/90)| ETA: 0.07|Total:0.02483|Binary:0.02348|Instance:0.00136|IoU:0.47480
Epoch 30 Step 50 |(50/90)| ETA: 0.07|Total:0.02481|Binary:0.02345|Instance:0.00136|IoU:0.51167
Epoch 30 Step 60 |(60/90)| ETA: 0.07|Total:0.02478|Binary:0.02342|Instance:0.00136|IoU:0.48469
Epoch 30 Step 70 |(70/90)| ETA: 0.07|Tota

Epoch 39 Step 10 |(10/90)| ETA: 0.07|Total:0.02298|Binary:0.02162|Instance:0.00136|IoU:0.46655
Epoch 39 Step 20 |(20/90)| ETA: 0.07|Total:0.02296|Binary:0.02160|Instance:0.00136|IoU:0.50462
Epoch 39 Step 30 |(30/90)| ETA: 0.07|Total:0.02293|Binary:0.02158|Instance:0.00136|IoU:0.55835
Epoch 39 Step 40 |(40/90)| ETA: 0.07|Total:0.02291|Binary:0.02156|Instance:0.00136|IoU:0.47324
Epoch 39 Step 50 |(50/90)| ETA: 0.07|Total:0.02289|Binary:0.02153|Instance:0.00136|IoU:0.54798
Epoch 39 Step 60 |(60/90)| ETA: 0.07|Total:0.02286|Binary:0.02151|Instance:0.00136|IoU:0.50374
Epoch 39 Step 70 |(70/90)| ETA: 0.07|Total:0.02284|Binary:0.02149|Instance:0.00136|IoU:0.47062
Epoch 39 Step 80 |(80/90)| ETA: 0.07|Total:0.02282|Binary:0.02147|Instance:0.00136|IoU:0.55918
Epoch 39 Step 90 |(90/90)| ETA: 0.04|Total:0.02280|Binary:0.02144|Instance:0.00136|IoU:0.57139
Epoch 40
Epoch 40 Step 10 |(10/90)| ETA: 0.07|Total:0.02277|Binary:0.02142|Instance:0.00136|IoU:0.46121
Epoch 40 Step 20 |(20/90)| ETA: 0.07|Tota

...

Epoch 58 Step 10 |(10/90)| ETA: 0.07|Total:0.01900|Binary:0.01765|Instance:0.00135|IoU:0.75583
Epoch 58 Step 20 |(20/90)| ETA: 0.07|Total:0.01898|Binary:0.01763|Instance:0.00135|IoU:0.77910
Epoch 58 Step 30 |(30/90)| ETA: 0.07|Total:0.01896|Binary:0.01761|Instance:0.00135|IoU:0.79911
Epoch 58 Step 40 |(40/90)| ETA: 0.07|Total:0.01894|Binary:0.01759|Instance:0.00135|IoU:0.77662
Epoch 58 Step 50 |(50/90)| ETA: 0.07|Total:0.01892|Binary:0.01756|Instance:0.00135|IoU:0.77178
Epoch 58 Step 60 |(60/90)| ETA: 0.07|Total:0.01889|Binary:0.01754|Instance:0.00135|IoU:0.81403
Epoch 58 Step 70 |(70/90)| ETA: 0.07|Total:0.01887|Binary:0.01752|Instance:0.00135|IoU:0.79872
Epoch 58 Step 80 |(80/90)| ETA: 0.07|Total:0.01885|Binary:0.01750|Instance:0.00135|IoU:0.78399
Epoch 58 Step 90 |(90/90)| ETA: 0.04|Total:0.01883|Binary:0.01748|Instance:0.00135|IoU:0.76849
Epoch 59
Epoch 59 Step 10 |(10/90)| ETA: 0.07|Total:0.01881|Binary:0.01745|Instance:0.00135|IoU:0.80474
Epoch 59 Step 20 |(20/90)| ETA: 0.07|Tota

Epoch 67 Step 60 |(60/90)| ETA: 0.07|Total:0.01719|Binary:0.01584|Instance:0.00135|IoU:0.82487
Epoch 67 Step 70 |(70/90)| ETA: 0.07|Total:0.01717|Binary:0.01582|Instance:0.00135|IoU:0.85544
Epoch 67 Step 80 |(80/90)| ETA: 0.07|Total:0.01715|Binary:0.01580|Instance:0.00135|IoU:0.83223
Epoch 67 Step 90 |(90/90)| ETA: 0.05|Total:0.01713|Binary:0.01578|Instance:0.00135|IoU:0.86952
Epoch 68
Epoch 68 Step 10 |(10/90)| ETA: 0.07|Total:0.01711|Binary:0.01576|Instance:0.00135|IoU:0.85535
Epoch 68 Step 20 |(20/90)| ETA: 0.07|Total:0.01710|Binary:0.01574|Instance:0.00135|IoU:0.85161
Epoch 68 Step 30 |(30/90)| ETA: 0.07|Total:0.01708|Binary:0.01572|Instance:0.00135|IoU:0.83699
Epoch 68 Step 40 |(40/90)| ETA: 0.07|Total:0.01706|Binary:0.01570|Instance:0.00135|IoU:0.85616
Epoch 68 Step 50 |(50/90)| ETA: 0.07|Total:0.01704|Binary:0.01569|Instance:0.00135|IoU:0.85289
Epoch 68 Step 60 |(60/90)| ETA: 0.07|Total:0.01702|Binary:0.01567|Instance:0.00135|IoU:0.85122
Epoch 68 Step 70 |(70/90)| ETA: 0.07|Tota

Epoch 77 Step 10 |(10/90)| ETA: 0.07|Total:0.01569|Binary:0.01434|Instance:0.00135|IoU:0.89598
Epoch 77 Step 20 |(20/90)| ETA: 0.07|Total:0.01567|Binary:0.01432|Instance:0.00135|IoU:0.89350
Epoch 77 Step 30 |(30/90)| ETA: 0.07|Total:0.01566|Binary:0.01430|Instance:0.00135|IoU:0.90598
Epoch 77 Step 40 |(40/90)| ETA: 0.07|Total:0.01564|Binary:0.01429|Instance:0.00135|IoU:0.88908
Epoch 77 Step 50 |(50/90)| ETA: 0.07|Total:0.01562|Binary:0.01427|Instance:0.00135|IoU:0.88331
Epoch 77 Step 60 |(60/90)| ETA: 0.07|Total:0.01561|Binary:0.01425|Instance:0.00135|IoU:0.86752
Epoch 77 Step 70 |(70/90)| ETA: 0.07|Total:0.01559|Binary:0.01424|Instance:0.00135|IoU:0.89945
Epoch 77 Step 80 |(80/90)| ETA: 0.07|Total:0.01558|Binary:0.01422|Instance:0.00135|IoU:0.90021
Epoch 77 Step 90 |(90/90)| ETA: 0.05|Total:0.01556|Binary:0.01421|Instance:0.00135|IoU:0.87470
Epoch 78
Epoch 78 Step 10 |(10/90)| ETA: 0.07|Total:0.01554|Binary:0.01419|Instance:0.00135|IoU:0.89242
Epoch 78 Step 20 |(20/90)| ETA: 0.07|Tota

Epoch 86 Step 60 |(60/90)| ETA: 0.07|Total:0.01441|Binary:0.01306|Instance:0.00135|IoU:0.90245
Epoch 86 Step 70 |(70/90)| ETA: 0.07|Total:0.01440|Binary:0.01304|Instance:0.00135|IoU:0.92119
Epoch 86 Step 80 |(80/90)| ETA: 0.07|Total:0.01438|Binary:0.01303|Instance:0.00135|IoU:0.91847
Epoch 86 Step 90 |(90/90)| ETA: 0.05|Total:0.01437|Binary:0.01302|Instance:0.00135|IoU:0.90206
Epoch 87
Epoch 87 Step 10 |(10/90)| ETA: 0.07|Total:0.01436|Binary:0.01300|Instance:0.00135|IoU:0.91328
Epoch 87 Step 20 |(20/90)| ETA: 0.07|Total:0.01434|Binary:0.01299|Instance:0.00135|IoU:0.88636
Epoch 87 Step 30 |(30/90)| ETA: 0.07|Total:0.01433|Binary:0.01298|Instance:0.00135|IoU:0.91351
Epoch 87 Step 40 |(40/90)| ETA: 0.07|Total:0.01432|Binary:0.01297|Instance:0.00135|IoU:0.89719
Epoch 87 Step 50 |(50/90)| ETA: 0.07|Total:0.01430|Binary:0.01295|Instance:0.00135|IoU:0.90313
Epoch 87 Step 60 |(60/90)| ETA: 0.07|Total:0.01429|Binary:0.01294|Instance:0.00135|IoU:0.92161
Epoch 87 Step 70 |(70/90)| ETA: 0.07|Tota

Epoch 96 Step 10 |(10/90)| ETA: 0.07|Total:0.01335|Binary:0.01200|Instance:0.00135|IoU:0.92488
Epoch 96 Step 20 |(20/90)| ETA: 0.07|Total:0.01334|Binary:0.01199|Instance:0.00135|IoU:0.92328
Epoch 96 Step 30 |(30/90)| ETA: 0.07|Total:0.01333|Binary:0.01198|Instance:0.00135|IoU:0.92164
Epoch 96 Step 40 |(40/90)| ETA: 0.07|Total:0.01332|Binary:0.01197|Instance:0.00135|IoU:0.91284
Epoch 96 Step 50 |(50/90)| ETA: 0.07|Total:0.01331|Binary:0.01195|Instance:0.00135|IoU:0.93938
Epoch 96 Step 60 |(60/90)| ETA: 0.07|Total:0.01329|Binary:0.01194|Instance:0.00135|IoU:0.92259
Epoch 96 Step 70 |(70/90)| ETA: 0.07|Total:0.01328|Binary:0.01193|Instance:0.00135|IoU:0.92267
Epoch 96 Step 80 |(80/90)| ETA: 0.07|Total:0.01327|Binary:0.01192|Instance:0.00135|IoU:0.93393
Epoch 96 Step 90 |(90/90)| ETA: 0.05|Total:0.01326|Binary:0.01191|Instance:0.00135|IoU:0.94491
Epoch 97
Epoch 97 Step 10 |(10/90)| ETA: 0.07|Total:0.01325|Binary:0.01190|Instance:0.00135|IoU:0.94081
Epoch 97 Step 20 |(20/90)| ETA: 0.07|Tota

In [7]:
# # 继续训练
# EPOCHES = 50
# file_dataset = './tusimple_0531/training'
# save_path = 'save_path'

# # 使用训练的数据进行可视化
# train_dataset_file = os.path.join(file_dataset, 'train.txt')

# train_dataset = LaneDataSet(train_dataset_file, 
#     transform=transforms.Compose([transforms.Resize((512,256),interpolation = 0)]))

# model = LaneNet().to(DEVICE)
# pth_path ='./save_path/50.pth'
# model.load_state_dict(torch.load(pth_path))
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle = False)
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# train(train_loader, model, optimizer, EPOCHES,save_path)

FileNotFoundError: [Errno 2] No such file or directory: './save_path/50.pth'